In [3]:
import json
import seaborn; seaborn.set()
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
raw_data = sc.textFile('hdfs://master:54310/fully_clean'). \
    map(lambda l: json.loads(l))

In [11]:
raw_data.count()

82073

In [13]:
feature_num = sc.textFile('hdfs://master:54310/word_idx'). \
    count()

In [14]:
from pyspark.mllib.linalg import Vectors

def transform_to_sparse(x: dict, feature_num: int):
    x['Features'] = Vectors.sparse(feature_num, x['Features'])
    return x

In [15]:
data = raw_data.map(lambda x: transform_to_sparse(x, feature_num)).persist()

In [18]:
label_count = data.flatMap(lambda x: x['Labels']). \
    countByValue()

In [47]:
sc.parallelize(label_count.items()). \
    sortBy(lambda lc: lc[1], ascending=True). \
    take(15)

[(218, 6),
 (227, 6),
 (128, 6),
 (134, 6),
 (91, 7),
 (240, 8),
 (62, 8),
 (87, 9),
 (112, 10),
 (353, 10),
 (22, 10),
 (143, 10),
 (306, 10),
 (89, 11),
 (127, 11)]

## exp 1

In [20]:
target_label = 81 # 1971 occur

In [22]:
from pyspark.mllib.regression import LabeledPoint

In [23]:
def transform_to_lpoint(x: dict, label: int):
    new_label = label in x['Labels']
    return LabeledPoint(new_label, x['Features'])

In [24]:
selection = data.map(lambda x: transform_to_lpoint(x, target_label))

In [27]:
train, test = selection.randomSplit([0.7, 0.3], seed=0)

In [30]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [31]:
lrm = LogisticRegressionWithSGD.train(train)

In [32]:
test_X = test.map(lambda row: row.features)
test_y = test.map(lambda row: row.label)

In [33]:
test_res = lrm.predict(test_X)

In [34]:
acc = test_y.zip(test_res).filter(lambda x: x[0] == x[1]).count() / test_y.count()

In [35]:
acc

0.974585230969421

## exp 2

In [58]:
target_label = 151 # 1551 occur

In [59]:
selection = data.map(lambda x: transform_to_lpoint(x, target_label))

In [60]:
train, test = selection.randomSplit([0.7, 0.3], seed=0)

In [61]:
train.filter(lambda row: row.label == 1).count()

1115

In [62]:
lrm2 = LogisticRegressionWithSGD.train(train)

In [63]:
test_X = test.map(lambda row: row.features)
test_y = test.map(lambda row: row.label)

In [64]:
test_res = lrm2.predict(test_X)
acc = test_y.zip(test_res).filter(lambda x: x[0] == x[1]).count() / test_y.count()

In [65]:
acc

0.9775943396226415

In [66]:
print(test_y.zip(test_res).filter(lambda x: x[0] == x[1]).count())
print(test_y.count())

24041
24592


## exp 3

In [48]:
target_label = 218 # 6 occur

In [49]:
selection = data.map(lambda x: transform_to_lpoint(x, target_label))

In [50]:
train, test = selection.randomSplit([0.7, 0.3], seed=0)

In [51]:
train.filter(lambda row: row.label == 1).count()

4

In [52]:
lrm = LogisticRegressionWithSGD.train(train)

In [53]:
test_X = test.map(lambda row: row.features)
test_y = test.map(lambda row: row.label)

In [54]:
test_res = lrm.predict(test_X)
acc = test_y.zip(test_res).filter(lambda x: x[0] == x[1]).count() / test_y.count()

In [55]:
acc

0.9999186727391022

In [56]:
print(test_y.zip(test_res).filter(lambda x: x[0] == x[1]).count())
print(test_y.count())

24590
24592


## exp 4

In [67]:
target_label = 81

In [68]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

In [105]:
def update_label(x: dict, label: int):
    x['Label'] = str(label in x['Labels'])
    del x['Labels']
    del x['Id']
    x['features'] = x['Features']
    del x['Features']
    return x

In [106]:
df = sqlContext.createDataFrame(data.map(lambda row: update_label(row, target_label)))

/home/hadoop/spark/python/pyspark/sql/context.py:234: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [107]:
df.columns

['Label', 'features']

In [112]:
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol = 'Label', outputCol = 'indexedLabel')
si_model = stringIndexer.fit(df)
td = si_model.transform(df)
td.columns

['Label', 'features', 'indexedLabel']

In [113]:
[train, test] = td.randomSplit([0.7, 0.3], seed=0)

In [114]:
def test_classification(train, test):
    rf = RandomForestClassifier(labelCol='indexedLabel')
    model = rf.fit(train)
    prediction_and_labels = model.transform(test).select('prediction', 'indexedLabel') \
        .map(lambda x: (x.prediction, x.label))
    return (prediction_and_labels.filter(lambda x: x[0] == x[1]), prediction_and_labels.count())

In [116]:
temp = test_classification(train, test)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 495.0 failed 4 times, most recent failure: Lost task 2.3 in stage 495.0 (TID 1935, 10.0.3.11): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1250, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'label' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoop/spark/python/pyspark/rdd.py", line 2352, in pipeline_func
    return func(split,

In [ ]:
temp[0]

In [ ]:
temp[1]